In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import tensorflow as tf
# tf.compat.v1.enable_eager_execution()

In [4]:
import numpy as np
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import best_rq

In [5]:
from malaya_speech.train.model.conformer.model import Model as ConformerModel

class Encoder:
    def __init__(self, config):
        self.config = config
        self.encoder = ConformerModel(**self.config)

    def __call__(self, x, input_mask, training = True):
        return self.encoder(x)
    
config_conformer = malaya_speech.config.conformer_base_encoder_config
config_conformer['subsampling']['type'] = 'none'
config_conformer['dropout'] = 0.0
encoder = Encoder(config_conformer)

In [6]:
# X = tf.placeholder(tf.float32, (None, None))
# X_len = tf.placeholder(tf.int32, (None,))

In [7]:
X = tf.placeholder(tf.float32, (None, None))
X_len = tf.placeholder(tf.int32, (None,))

In [8]:
cfg = best_rq.Best_RQConfig(dropout=0.0,
                                attention_dropout=0.0,
                                encoder_layerdrop=0.0,
                                dropout_input=0.0,
                                dropout_features=0.0)

In [9]:
model = best_rq.Model(cfg, encoder)

In [10]:
out = model(X, padding_mask = X_len)
out

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


{'x': <tf.Tensor 'best_rq/dense_1/BiasAdd:0' shape=(?, ?, 8192) dtype=float32>,
 'padding_mask': <tf.Tensor 'best_rq/LogicalNot:0' shape=(?, ?) dtype=bool>,
 'y': <tf.Tensor 'best_rq/unmasked_features:0' shape=(?, ?, 512) dtype=float32>,
 'onehot': <tf.Tensor 'best_rq/Reshape_1:0' shape=(?, ?) dtype=int64>}

In [11]:
masks = tf.cast(tf.math.logical_not(out['padding_mask']), tf.float32)

In [12]:
cost = tf.contrib.seq2seq.sequence_loss(logits = out['x'],
                                                     targets = out['onehot'],
                                                     weights = masks)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [14]:
y, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav')
len(y) / sr

3.518625

In [15]:
y1 = np.zeros(shape = (40000))

In [16]:
from malaya_speech.utils.padding import sequence_1d

In [17]:
batch_x, batch_len = sequence_1d([y, y1], return_len = True)

In [18]:
%%time

o = sess.run([out, cost], feed_dict = {X: batch_x, X_len: batch_len})

CPU times: user 4.75 s, sys: 570 ms, total: 5.32 s
Wall time: 3.57 s


In [19]:
o[1]

9.126385